http://matrix.skku.ac.kr/math4AI-tools/cosine_similarity/

In [14]:
import numpy as np

def cos_similarity(v1,v2):
    dot_product = np.dot(v1,v2)
    l2_norm = (np.sqrt(sum(np.square(v1)))*np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm
    return similarity

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

doc_list = ['if you take the blue pill, the story ends' ,
            'if you take the red pill, you stay in Wonderland',
            'if you take the red pill, I show you how deep the rabbit hole goes']

Tfidf_vect_simple = TfidfVectorizer()
feature_vect_simple = Tfidf_vect_simple.fit_transform(doc_list)
#feature_vect_simple.toarray()
feature_vect_simple.shape

(3, 18)

In [17]:
feature_vect_dense = feature_vect_simple.todense()
vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

similarity_simple = cos_similarity(vect1, vect2)
print(f'코사인 유사도:{similarity_simple:.4f}')

코사인 유사도:0.4021


In [13]:
from sklearn.metrics.pairwise import cosine_similarity
sim_pair = cosine_similarity(feature_vect_simple[0], feature_vect_simple)
sim_pair

array([[1.        , 0.40207758, 0.40425045]])

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
sim_pair1 = cosine_similarity(feature_vect_simple[0], feature_vect_simple[1:])
sim_pair1

array([[0.40207758, 0.40425045]])

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
sim_pair1 = cosine_similarity(feature_vect_simple, feature_vect_simple)
sim_pair1

array([[1.        , 0.40207758, 0.40425045],
       [0.40207758, 1.        , 0.45647296],
       [0.40425045, 0.45647296, 1.        ]])

[과제] Opinion Review 데이터 세트를 이용한 문서 유사도를 측정하세요

In [ ]:
def visualize_silhouette(cluster_lists, X_features, max_iter): 
    
    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math
    
    n_cols = 4
    n_rows = int(np.ceil(len(cluster_lists)/4))
    
    fig = plt.figure(figsize = (25, 8*n_rows))
    plt.subplots_adjust(hspace = 0.35)
#     plt.subplots_adjust(left=0.125, bottom=0.1,  right=0.9, top=0.9, wspace=0.2, hspace=0.35)  # 서브플롯 공간 띄우기
    for ind, n_cluster in enumerate(cluster_lists):
        clusterer = KMeans(n_clusters = n_cluster, max_iter = max_iter, random_state = 0)
        cluster_labels = clusterer.fit_predict(X_features)
        
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        
        y_lower = 10
        ax = fig.add_subplot(n_rows, n_cols, ind+1)
        ax.set_title(f'\
            max_iter : {str(max_iter)} \n\n \
            Number of Cluster : {str(n_cluster)} \n \
            Silhouette Score : {str(round(sil_avg,3))}')
        ax.set_xlabel("The silhouette coefficient values")
        ax.set_ylabel("Cluster label")
        ax.set_xlim([-0.1, 1])
        ax.set_ylim([0, X_features.shape[0] + (n_cluster + 1) * 10])
        ax.set_yticks([])  # Clear the yaxis labels / ticks
        ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
        # 클러스터링 갯수별로 fill_betweenx()형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()
            
            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            color = cm.nipy_spectral(float(i) / n_cluster)
            ax.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10
            
        ax.axvline(x=sil_avg, color="red", linestyle="--")